In [1]:
import numpy as np
import laser_hockey_env as lh
import gym
import tensorflow as tf
from importlib import reload

In [2]:
#np.set_printoptions(suppress=True)
np.set_printoptions(threshold = 1e5)

# Normal Game Play

In [3]:
reload(lh)

<module 'laser_hockey_env' from 'C:\\Users\\ThinkPad\\Desktop\\Informatik\\3. Semester\\Reinforcement Learning\\final project\\laser-hockey-env-master\\Prioritized_Replay_DQN\\laser_hockey_env.py'>

In [4]:
env = lh.LaserHockeyEnv(mode=1)

have a look at the initialization condition: alternating who starts and are random in puck position

In [5]:
obs = env.reset()
#obs_agent2 = env.obs_agent_two()
#env.render()
obs

array([-6.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  7.90865898, -0.08248997,  0.        ,  0.        ,
        0.        ,  0.        , -3.50483894, -1.03803205,  0.        ,
        0.        ])

# Model Recovery Test

In [5]:
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('PR_DQN_model.ckpt.meta')
    saver.restore(sess, 'PR_DQN_model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    
    for op in graph.get_operations():
        print(op.name)
    
    #Qs = tf.get_collection('pred_Qs')[0]
    pred = graph.get_operation_by_name('DQN_with_prioritized_replay/eval_net/l3/add').outputs[0]
    print(pred)
    #Qs = graph.get_tensor_by_name('DQNAgent/Q_target/dense_1/kernel:0')
    
    #state = tf.get_collection('s')
    
    state = graph.get_operation_by_name('DQN_with_prioritized_replay/s').outputs[0]
    print(state)

INFO:tensorflow:Restoring parameters from PR_DQN_model.ckpt
DQN_with_prioritized_replay/s
DQN_with_prioritized_replay/Q_target
DQN_with_prioritized_replay/IS_weights
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal/shape
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal/mean
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal/stddev
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal/RandomStandardNormal
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal/mul
DQN_with_prioritized_replay/eval_net/l1/w1/Initializer/random_normal
DQN_with_prioritized_replay/eval_net/l1/w1
DQN_with_prioritized_replay/eval_net/l1/w1/Assign
DQN_with_prioritized_replay/eval_net/l1/w1/read
DQN_with_prioritized_replay/eval_net/l1/b1/Initializer/Const
DQN_with_prioritized_replay/eval_net/l1/b1
DQN_with_prioritized_replay/eval_net/l1/b1/Assign
DQN_with_prioritized_replay/eval_net/l1/b1/read
DQN_with_prioritized_replay/eval_n

# For PR_DQN Agents

In [8]:
reload(lh)
env = lh.LaserHockeyEnv(mode=0)

In [ ]:
AGAINST_HUMAN = False
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('PR_DQN_model.ckpt.meta')
    saver.restore(sess, 'PR_DQN_model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    
    playerHuman = lh.HumanOpponent(env=env, player=2)
    playerComputer = lh.BasicOpponent()
    
    pred = graph.get_operation_by_name('DQN_with_prioritized_replay/target_net/l3/add').outputs[0]    
    state = graph.get_operation_by_name('DQN_with_prioritized_replay/s_').outputs[0]
    
    win = 0
    die = 0
    loss = 0
    round1 = 1000
    for _ in range(round1):
        s = env.reset()
        for _ in range(500):
            reward = 0
            env.render()
            qs = np.argmax(sess.run(pred, {state: np.asarray(s).reshape(-1,16)}))
            a1 = env.discrete_to_continous_action(qs)
            if AGAINST_HUMAN:
                a2 = playerHuman.act(env.obs_agent_two())
            else:
                a2 = playerComputer.act(env.obs_agent_two())    
            obs, r, d, info = env.step(np.hstack([a1,a2]))
            s = obs
            reward += r
            #print (info)
            if d:
                winner = info['winner']
                if winner == 1:
                    win += 1
                elif winner == 0:
                    die += 1
                else:
                    loss += 1
                break

        #print ('total points: ', reward)
    print('win rate in test:', win / (round1))
    print('die rate in test:',die/ (round1))
    print('loss rate in test:',loss/(round1))
env.close()#

INFO:tensorflow:Restoring parameters from PR_DQN_model.ckpt
Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 1 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 2 scored
Player 1 scored
Player 1 scored
Player 1 scored
Player 2 scored
Player 1 scored
Player 2 scored
Player 2 scored
Player 2 score

In [24]:
env.close()

"info" dict contains useful proxy rewards and winning information

Winner == 0: draw

Winner == 1: you (left player)

Winner == -1: opponent wins (right player)

# Hand-crafted Opponent

In [166]:
reload(lh)

<module 'laser_hockey_env' from 'C:\\Users\\ThinkPad\\Desktop\\Informatik\\3. Semester\\Reinforcement Learning\\final project\\laser-hockey-env-master\\laser_hockey_env.py'>

In [167]:
env = lh.LaserHockeyEnv()

In [43]:
o = env.reset()
env.render()

In [169]:
player1 = lh.BasicOpponent()
player2 = lh.BasicOpponent()

In [170]:
obs_buffer = []

In [190]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(6000):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_buffer.append(obs)
    obs_agent2 = env.obs_agent_two()
    if d: break

In [36]:
obs_buffer = np.asarray(obs_buffer)

In [37]:
np.mean(obs_buffer,axis=0)

array([-5.60615518, -0.93944872,  0.02153415, -0.15174563, -0.21507722,
       -0.01274596,  6.03210314, -0.91700226, -0.0435948 ,  0.08250999,
       -0.45369915, -0.01376339,  0.03930392, -1.95906703,  0.34407332,
       -0.42500431])

In [38]:
np.std(obs_buffer,axis=0)

array([ 1.9823213 ,  1.70992644,  0.17918739,  3.50116077,  5.45221764,
        0.79510197,  1.89944108,  1.53295154,  0.17746532,  3.52693113,
        5.35313172,  0.81996932,  3.81635594,  2.37721153, 15.28987793,
        4.71857464])

In [39]:
scaling = [ 1.0,  1.0 , 3.14, 4.0, 4.0, 2.0,  
            1.0,  1.0,  3.14, 4.0, 4.0, 2.0,  
            2.0, 2.0, 10.0, 10.0]

# Human Opponent

In [40]:
import time

In [41]:
env = lh.LaserHockeyEnv()

In [42]:
o = env.reset()
env.render()

True

In [43]:
player1 = lh.HumanOpponent(env=env, player=1)
player2 = lh.BasicOpponent()

Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s


In [50]:
obs = env.reset()
time.sleep(1)
obs_agent2 = env.obs_agent_two()
for _ in range(6000):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    print (info)
    if d: break

{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_closeness_to_puck': -0.0036225496896808897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
{'winner': 0, 'reward_cl

{'winner': 0, 'reward_closeness_to_puck': -0.0010327091614482367, 'reward_touch_puck': 1.0, 'reward_puck_direction': 0.0009404952049255371}
{'winner': 0, 'reward_closeness_to_puck': -0.0012102154815832168, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0009395546913146973}
{'winner': 0, 'reward_closeness_to_puck': -0.0013790704436083335, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0009386151313781739}
{'winner': 0, 'reward_closeness_to_puck': -0.001538090714723554, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0009376765251159668}
{'winner': 0, 'reward_closeness_to_puck': -0.0016867431120614713, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0009367388725280763}
{'winner': 0, 'reward_closeness_to_puck': -0.0018248189827129897, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.000935802173614502}
{'winner': 0, 'reward_closeness_to_puck': -0.0019522789615913014, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0009348664283752441}
{'winner': 0, 'reward_

{'winner': 0, 'reward_closeness_to_puck': -0.013229034050107138, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.00018159109354019167}
{'winner': 0, 'reward_closeness_to_puck': -0.013089186761756494, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.00018140950202941896}
{'winner': 0, 'reward_closeness_to_puck': -0.01293245544043022, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.00018122808933258058}
{'winner': 0, 'reward_closeness_to_puck': -0.012759112071614965, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0001810468673706055}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0001808658242225647}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.00018068495988845826}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.00018050427436828615}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck

{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.059230327606201e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.054171085357666e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.049117207527161e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.044068098068238e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.039024353027344e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.033985376358033e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.028951168060303e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 5.0239223241806034e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'r

{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.77543330192566e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.766657710075379e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.757891058921814e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.749133348464966e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.740384578704834e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.73164415359497e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.722912669181824e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 8.714190125465394e-05}
{'winner': 0, 'reward_closeness_to_puck': 0, 'rewa

{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.811474055051804e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.80366286635399e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.79585912823677e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.78806358575821e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.780275493860245e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.772495597600937e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.764723151922227e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 7.756958156824112e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'rewar

{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.2218022495508197e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.219580486416817e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.217360958456993e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.2151436656713487e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.212928608059883e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.210715785622597e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.2085050120949745e-06}
{'winner': 0, 'reward_closeness_to_puck': 0, 'reward_touch_puck': 0.0, 'reward_puck_direction': 2.2062964737415314e-06}
{'winner': 0, 'reward_closeness_to_puck': 0,